In [162]:
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'
require 'optim'

In [2]:
aroraname = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv'
wordfile = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/total_corpus_smry.csv'
nuggets = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/aurora_nuggets_numtext.csv'

In [3]:
m = csvigo.load({path = aroraname, mode = "large"})
w = csvigo.load({path = wordfile, mode = "large"})
q = csvigo.load({path = nuggets, mode = "large"})

<csv>	parsing file: /Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/2012_aurora_shooting_first_sentence_numtext.csv	


<csv>	parsing done	


In [66]:
N = 1000
K = 100
embed_dim = 6

In [6]:
function split(pString)
   local pPattern = " "
   local Table = {}  -- NOTE: use {n = 0} in Lua-5.0
   local fpat = "(.-)" .. pPattern
   local last_end = 1
   local s, e, cap = pString:find(fpat, 1)
   while s do
      if s ~= 1 or cap ~= "" then
     table.insert(Table, cap)
      end
      last_end = e + 1
      s, e, cap = pString:find(fpat, last_end)
   end
   if last_end <= #pString then
      cap = pString:sub(last_end)
      table.insert(Table, cap)
   end
   return Table
end

function grabKtokens(x, K)
    local tmp = {}
    for k, v in pairs(x) do
        if k <= K then
            tmp[k] = v
        end
        if k > K then
            return tmp
        end
    end
    return tmp
end

function grabNsamples(x, N, K)
    local out = {}
    for k,v in pairs(m) do
        if k > 1 then
            out[k-1] = grabKtokens(split(x[k][1]), K)
        end
        if (k % (N+1))==0 then
            print(N,'elements read out of ', #x)
            break
        end
    end
    return out
end

In [50]:
function padZeros(x, maxlen)
    local out = {}
    for k, v in pairs(x) do
        tmp = {}
        if #v <  maxlen then
            for i=1, maxlen do
                if i <= (maxlen - #v) then
                    tmp[i] = 0
                else 
                    tmp[i] = v[i - (maxlen-#v)]
                end
            end
        else 
            tmp = v
        end
        out[k] = tmp
    end
    return out
end

In [78]:
--- Extracting N samples
out = grabNsamples(m, N, K)

mxl = 0
for k,v in pairs(out) do
    mxl = math.max(mxl, #v)
end

--- getting the length of the dictionary
vocab_size = 0
for k,v in pairs(out) do
    vocab_size = math.max(vocab_size, math.max(table.unpack(v)))
    if (k % N)==0 then
        print(k,'elements read out of ', #m)
        break
    end
end

--- Padding the data by the maximum length
xs = padZeros(out, mxl)

1000	elements read out of 	12510	
1000	elements read out of 	12510	


In [159]:
--- labels = torch.Tensor(torch.rand(#out)*20.):reshape(#out,1)
labels = torch.Tensor(torch.round(torch.rand(#out))):reshape(#out,1)

In [149]:
--- This is the correct format to input it
input = torch.LongTensor(xs)

In [144]:
LT = nn.LookupTableMaskZero(vocab_size, embed_dim)

-- For batch inputs, it's a little easier to start with sequence-length x batch-size tensor, so we transpose songData
myDataT = input:t()
batchLSTM = nn.Sequential()
batchLSTM:add(LT) -- will return a sequence-length x batch-size x embedDim tensor
batchLSTM:add(nn.SplitTable(1, embed_dim)) -- splits into a sequence-length table with batch-size x embedDim entries
-- print(batchLSTM:forward(myDataT)) -- sanity check
-- now let's add the LSTM stuff
batchLSTM:add(nn.Sequencer(nn.LSTM(embed_dim, embed_dim)))
batchLSTM:add(nn.SelectTable(-1)) -- selects last state of the LSTM
batchLSTM:add(nn.Linear(embed_dim, 1)) -- map last state to a score for classification
batchLSTM:add(nn.Linear(1, 1)) -- convert score to a probability
--- batchLSTM:add(nn.Sigmoid()) -- convert score to a probability
myPreds = batchLSTM:forward(myDataT)
print(#myPreds)

-- we can now call :backward() as follows
bceCrit = nn.MSECriterion()
loss = bceCrit:forward(myPreds, labels)
dLdPreds = bceCrit:backward(myPreds, labels)
batchLSTM:backward(myDataT, dLdPreds)
print(loss)

 1000
    1
[torch.LongStorage of size 2]



155.0838741741	


In [153]:
inputs = input

In [154]:
target = labels

In [155]:
criterion = nn.MSECriterion()

In [165]:
epochs = 10
confusion = optim.ConfusionMatrix({0,1})

In [21]:
trainSet = {inputs}
trainSet.ivocab = vocab_size

In [26]:
lookupDim = 10

In [27]:
model = nn.Sequential()

-- Transpose, such that input is seqLen x batchSize
model:add(nn.Transpose({1,2}))

-- LookupTable
local lookup = nn.LookupTableMaskZero(trainSet.ivocab, lookupDim)
model:add(lookup)
if lookupDropout ~= 0 then model:add(nn.Dropout(lookupDropout)) end

-- Recurrent layers
local inputSize = lookupDim
for i, hiddenSize in ipairs(hiddenSizes) do
  local rnn = nn.SeqLSTM(inputSize, hiddenSize)
  rnn.maskzero = true
  model:add(rnn)
  if dropouts[i] ~= 0 and dropouts[i] ~= nil then
     model:add(nn.Dropout(dropouts[i]))
  end
  inputSize = hiddenSize 
end
model:add(nn.Select(1, -1))

-- Output Layer
model:add(nn.Linear(hiddenSizes[#hiddenSizes], #classes))
model:add(nn.LogSoftMax())

[string "model = nn.Sequential()..."]:13: bad argument #1 to 'ipairs' (table expected, got nil)
stack traceback:
	[C]: in function 'ipairs'
	[string "model = nn.Sequential()..."]:13: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x01016d7b90: 

In [ ]:
for epoch=1, epochs do
   -- Single training epoch
   trainLoss = 0
   confusion:zero()
   model:training()
   for i, inputs, targets in myDataT:sampleiter(batchSize, epochSize) do
      xlua.progress(i, epochSize)
      trainInputs:resize(inputs:size()):copy(inputs)
      trainTargets:resize(targets:size()):copy(targets)

      local feval = function()
         gradParameters:zero()

         -- Forward
         local outputs = model:forward(trainInputs)
         local f = criterion:forward(outputs, trainTargets)
         trainLoss = trainLoss + f

         -- Backward
         local df_do = criterion:backward(outputs, trainTargets)
         model:backward(trainInputs, df_do)

         confusion:batchAdd(conOutputs, conTargets)
         return f, gradParameters
      end
      optimMethod(feval, parameters, optimState)
   end
   confusion:updateValids()
   if best_train_accu < confusion.totalValid then
      print("Best train accuracy: ".. best_train_accu ..
                  " current accu: ".. confusion.totalValid)
      best_train_accu = confusion.totalValid
      --best_train_model = model:clone()
   end
end

In [125]:
preds = {}
for i=1, myPreds:size()[1] do
    preds[i] = (myPreds[i][1] > 0.5) and 1 or 0 --- lua is stupid
end

In [43]:
for k,v in pairs(xs[1]) do return v end

In [67]:
function unpackZeros(x) 
    local out = {}
    local c = 1
    for k,v in pairs(x) do
        if v~=0 then
            out[c] = v
            c = c +1
        end
    end
    return out
end
function buildPredSummary(preds, xs) 
    local predsummary = {}
    c = 1
    for i=1, #xs do
        if preds[i]== 1 then
            predsummary[c] = unpackZeros(xs[i])
            c = c + 1
        end
    end
    return predsummary
end

In [72]:
predsummary

{
  1 : 
    {
      1 : 1381
      2 : 479
      3 : 1135
      4 : 47
      5 : 81
    }
  2 : 
    {
      1 : 59
      2 : 23
      3 : 66
      4 : 29
      5 : 535
    }
  3 : 
    {
      1 : 111
      2 : 1591
      3 : 48
      4 : 57
      5 : 1135
      6 : 2206
    }
  4 : 
    {
      1 : 142
      2 : 1464
      3 : 437
      4 : 1591
      5 : 48
      6 : 57
      7 : 1135
      8 : 2206
    }
  5 : 
    {
      1 : 1464
      2 : 467
      3 : 57
      4 : 2206
      5 : 48
      6 : 1
      7 : 111
      8 : 1591
      9 : 1135
      10 : 48
      11 : 1188
      12 : 48
      13 : 81
      14 : 479
      15 : 1
      16 : 57
      17 : 1135
      18 : 57
      19 : 299
    }
  6 : 
    {
      1 : 24
      2 : 57
      3 : 23
      4 : 57
      5 : 1
      6 : 1595
      7 : 28
      8 : 71
      9 : 57
      10 : 467
    }
  7 : 
    {
      1 : 2990
      2 : 62
      3 : 24
      4 : 138
      5 : 67
      6 : 57
      7 : 192
      8 : 24
      9 : 401
      10 :

   3 : 177
      4 : 147
      5 : 81
      6 : 57
      7 : 1135
      8 : 111
      9 : 299
    }
  12 : 
    {
      1 : 277
      2 : 47
      3 : 57
      4 : 151
      5 : 119
      6 : 1737
      7 : 177
      8 : 67
      9 : 81
    }
}
